In [1]:
!pip install beautifulsoup4
!pip install geopy

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


## 1. Getting data from wikipedia

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

## 2. Parsing data with Beautiful Soup

In [4]:
data_new = BeautifulSoup(data, 'html.parser')

In [5]:
PostalCode = []
Borough =[]
Neighborhood = []

data_new.head()

[<meta charset="utf-8"/>,
 <title>List of postal codes of Canada: M - Wikipedia</title>,
 <script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"70903ad1-cd3c-46aa-8385-fd98ec3ea5a3","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgPageContentLanguage":"en","wgPageContentModel":"wik

In [6]:
for i in data_new.find('table').find_all('tr'):
    cells = i.find_all('td')
    if(len(cells)>0):
        PostalCode.append(cells[0].text.rstrip('\n'))
        Borough.append(cells[1].text.rstrip('\n'))
        Neighborhood.append(cells[2].text.rstrip('\n'))

## 3. Creating a dictionary and converting to pd df

In [7]:
dict_1 = {'Postal Code':PostalCode,'Borough':Borough,'Neighborhood':Neighborhood}

In [8]:
tor_df = pd.DataFrame(data=dict_1)

In [9]:
tor_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
cols = ['Postal Code','Borough','Neighborhood']
tor_df[cols] = tor_df[cols].replace({'\n':''}, regex=True)

In [11]:
tor_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## 5. Dropping rows with unassigned Boroughs

In [12]:
tor_df.drop(tor_df[tor_df['Borough']=='Not assigned'].index, inplace = True) 

In [13]:
tor_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
tor_df=tor_df.reset_index()

In [15]:
del tor_df['index']

In [16]:
tor_df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## 6. Giving Neighborhood's the same name as Borough if not assigned 

In [17]:
for i, j in tor_df.iterrows():
    if j["Neighborhood"] == "Not assigned":
        j["Neighborhood"] = j["Borough"]
        
tor_df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [18]:
tor_df['Postal Code'].value_counts()

M4G    1
M4M    1
M1L    1
M1W    1
M1K    1
      ..
M2L    1
M6H    1
M6N    1
M3L    1
M9A    1
Name: Postal Code, Length: 103, dtype: int64

## 7. Groupby to remove duplicates of PostalCode

In [19]:
tor_df_grouped = tor_df.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
tor_df_grouped

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


## 8. Clean Data

In [20]:
tor_df_grouped['Postal Code'].value_counts()

M4G    1
M5B    1
M1J    1
M3C    1
M7A    1
      ..
M9M    1
M1X    1
M6H    1
M6N    1
M9A    1
Name: Postal Code, Length: 103, dtype: int64

In [21]:
column_names = ["Postal Code", "Borough", "Neighborhood"]
test = pd.DataFrame(columns=column_names)

test_1 = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_1:
    test = test.append(tor_df_grouped[tor_df_grouped["Postal Code"]==postcode], ignore_index=True)
    
test

,Postal Code,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [22]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
tor_df_merged = tor_df_grouped.merge(coordinates, on="Postal Code", how="left")
tor_df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [24]:
tor_df_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [25]:
column_names = ["Postal Code", "Borough", "Neighborhood","Latitude","Longitude"]
test = pd.DataFrame(columns=column_names)

test_1 = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_1:
    test = test.append(tor_df_merged[tor_df_merged["Postal Code"]==postcode], ignore_index=True)
    
test

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


## 9. Plot Toronto on the map

In [26]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [27]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor_df_merged['Latitude'], tor_df_merged['Longitude'], tor_df_merged['Borough'], tor_df_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 10. Extract only Toronto Neighborhoods

In [28]:
bor_names = list(tor_df_merged.Borough.unique())

bor_with_tor = []

for i in bor_names:
    if "toronto" in i.lower():
        bor_with_tor.append(i)
        
tor_df_new = tor_df_merged[tor_df_merged['Borough'].isin(bor_with_tor)].reset_index(drop=True)
tor_df_new.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


## 11. Use Foursquare ID to get venues close to toronto neighborhood

In [29]:
CLIENT_ID = '1C0SX4WX4MWTBKJQA5V0KWTDPNMAYOBJQQDWINWK0TH3Z0NT' # your Foursquare ID
CLIENT_SECRET = 'RSTB1BDY1DYRHQAMW4T5W3XEYNTO5A1IZPWB0HEQVSYNLHWX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1C0SX4WX4MWTBKJQA5V0KWTDPNMAYOBJQQDWINWK0TH3Z0NT
CLIENT_SECRET:RSTB1BDY1DYRHQAMW4T5W3XEYNTO5A1IZPWB0HEQVSYNLHWX


In [30]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(tor_df_new['Latitude'], tor_df_new['Longitude'], tor_df_new['Postal Code'], tor_df_new['Borough'], tor_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [31]:
df_venue = pd.DataFrame(venues)
df_venue.columns = ['Postal Code', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
df_venue.shape


(1623, 9)

## 12.Groupby venues for each neighborhood

In [32]:
df_venue.groupby(["Neighborhood"]).count()

,Postal Code,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,,,
Berczy Park,58,58,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14,14,14
Central Bay Street,65,65,65,65,65,65,65,65
Christie,17,17,17,17,17,17,17,17
Church and Wellesley,76,76,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100,100,100
Davisville,35,35,35,35,35,35,35,35


In [33]:
print('There are {} uniques categories.'.format(len(df_venue['VenueCategory'].unique())))

There are 236 uniques categories.


## 13. Analyse each neighborhood

In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(df_venue[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['Postal Code'] = df_venue['Postal Code'] 
toronto_onehot['Borough'] = df_venue['Borough'] 
toronto_onehot['Neighborhoods'] = df_venue['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()


(1623, 239)


,Postal Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
toronto_grouped = toronto_onehot.groupby(["Neighborhoods","Postal Code","Borough"]).mean().reset_index()
toronto_grouped

,Neighborhoods,Postal Code,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,M5E,Downtown Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",M6K,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",M7Y,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,Downtown Toronto,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.071429,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,M5G,Downtown Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000,0.015385
5,Christie,M6G,Downtown Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,M4Y,Downtown Toronto,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.026316
7,"Commerce Court, Victoria Hotel",M5L,Downtown Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,M4S,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,M4P,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##  14. Gyms in Toronto and CLustering

In [38]:
"Gym" in df_venue['VenueCategory'].unique()

True

In [39]:
len(toronto_grouped[toronto_grouped["Gym"] > 0])

13

In [43]:
toronto_gyms = toronto_grouped[["Neighborhoods","Gym"]]
toronto_gyms.head()

,Neighborhoods,Gym
0,Berczy Park,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.045455
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.000000


In [44]:
from sklearn.cluster import KMeans
clusters = 3

Km = toronto_gyms.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=1)
kmeans.fit_transform(Km)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 0, 1, 1, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1],
      dtype=int32)

In [48]:
tor_merged = toronto_gyms.copy()
tor_merged["Cluster Labels"] = kmeans.labels_

In [49]:
tor_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
tor_merged.head(5)

,Neighborhood,Gym,Cluster Labels
0,Berczy Park,0.000000,1
1,"Brockton, Parkdale Village, Exhibition Place",0.045455,0
2,"Business reply mail Processing Centre, South C...",0.000000,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,1
4,Central Bay Street,0.000000,1


In [52]:
tor_merged = tor_merged.join(df_venue.set_index("Neighborhood"), on="Neighborhood")

print(tor_merged.shape)
tor_merged.head()

(1623, 11)


,Neighborhood,Gym,Cluster Labels,Postal Code,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Berczy Park,0.0,1,M5E,Downtown Toronto,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,1,M5E,Downtown Toronto,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,1,M5E,Downtown Toronto,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,1,M5E,Downtown Toronto,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,1,M5E,Downtown Toronto,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [53]:
tor_merged.sort_values(["Cluster Labels"], inplace=True)
tor_merged.head()

,Neighborhood,Gym,Cluster Labels,Postal Code,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
11,"First Canadian Place, Underground city",0.04,0,M5X,Downtown Toronto,43.648429,-79.38228,Pi Co.,43.648651,-79.385874,Pizza Place
11,"First Canadian Place, Underground city",0.04,0,M5X,Downtown Toronto,43.648429,-79.38228,Brick Street Bakery,43.648815,-79.380605,Bakery
11,"First Canadian Place, Underground city",0.04,0,M5X,Downtown Toronto,43.648429,-79.38228,Pizzeria Libretto,43.648334,-79.385111,Pizza Place
11,"First Canadian Place, Underground city",0.04,0,M5X,Downtown Toronto,43.648429,-79.38228,Maman,43.648309,-79.382253,Café
11,"First Canadian Place, Underground city",0.04,0,M5X,Downtown Toronto,43.648429,-79.38228,Hy's Steakhouse,43.649505,-79.382919,Steakhouse


In [56]:
map_new = folium.Map(location=[latitude, longitude], zoom_start=10)
# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(tor_merged['BoroughLatitude'], tor_merged['BoroughLongitude'], tor_merged['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_new)
       
map_new

## 15. Examine Clusters

### Cluster 1

In [58]:
tor_merged.loc[(tor_merged['Cluster Labels'] ==0) & (tor_merged['VenueCategory'] == 'Gym') ]

,Neighborhood,Gym,Cluster Labels,Postal Code,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
11,"First Canadian Place, Underground city",0.040000,0,M5X,Downtown Toronto,43.648429,-79.382280,Equinox Bay Street,43.648100,-79.379989,Gym
11,"First Canadian Place, Underground city",0.040000,0,M5X,Downtown Toronto,43.648429,-79.382280,Toronto Athletic Club,43.647360,-79.381177,Gym
7,"Commerce Court, Victoria Hotel",0.040000,0,M5L,Downtown Toronto,43.648198,-79.379817,GoodLife Fitness Toronto Street Women's Only,43.650400,-79.376700,Gym
8,Davisville,0.057143,0,M4S,Central Toronto,43.704324,-79.388790,GoodLife Fitness Toronto Mount Pleasant and Da...,43.700802,-79.386417,Gym
8,Davisville,0.057143,0,M4S,Central Toronto,43.704324,-79.388790,Crossfit Metric,43.707480,-79.389857,Gym
11,"First Canadian Place, Underground city",0.040000,0,M5X,Downtown Toronto,43.648429,-79.382280,The Cambridge Club,43.651663,-79.383075,Gym
11,"First Canadian Place, Underground city",0.040000,0,M5X,Downtown Toronto,43.648429,-79.382280,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym
31,Stn A PO Boxes,0.020619,0,M5W,Downtown Toronto,43.646435,-79.374846,Equinox Bay Street,43.648100,-79.379989,Gym
31,Stn A PO Boxes,0.020619,0,M5W,Downtown Toronto,43.646435,-79.374846,GoodLife Fitness Toronto Street Women's Only,43.650400,-79.376700,Gym
25,"Richmond, Adelaide, King",0.042553,0,M5H,Downtown Toronto,43.650571,-79.384568,Equinox Bay Street,43.648100,-79.379989,Gym


### CLuster 2

In [59]:
tor_merged.loc[(tor_merged['Cluster Labels'] ==1) & (tor_merged['VenueCategory'] == 'Gym') ]

,Neighborhood,Gym,Cluster Labels,Postal Code,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
37,"Toronto Dominion Centre, Design Exchange",0.01,1,M5K,Downtown Toronto,43.647177,-79.381576,Equinox Bay Street,43.648100,-79.379989,Gym
13,"Garden District, Ryerson",0.01,1,M5B,Downtown Toronto,43.657162,-79.378937,GoodLife Fitness Toronto Bell Trinity Centre,43.653436,-79.382314,Gym


### Cluster 3

In [60]:
tor_merged.loc[(tor_merged['Cluster Labels'] ==2) & (tor_merged['VenueCategory'] == 'Gym') ]

,Neighborhood,Gym,Cluster Labels,Postal Code,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
9,Davisville North,0.125,2,M4P,Central Toronto,43.712751,-79.390197,Gym,43.713126,-79.393537,Gym


### Understanding and decisions made

Most of the gyms are located in cluster 0 around the areas of Richmond, St James Town and First Canadian place, the best place to open a gym would be in the areas of cluster 2 around Davisville in Central Toronto